## Requirements
The code files require:
1. [Arduino IDE](https://www.arduino.cc/en/main/software)
2. [Python 3](https://www.python.org/downloads/)
3. [Numpy](https://scipy.org/install.html)
4. [OpenCV](https://opencv.org/releases/)
5. [Serial](https://pypi.python.org/pypi/pyserial)
6. [Matplotlib](https://matplotlib.org/users/installing.html)
7. [The Python Standard Library (for time, subprocess, csv, threading, os, shutil)](https://docs.python.org/3/library/)

Before trying the required software and libraries, run the following from terminal:
`$ sudo apt update`

`$ sudo apt upgrade`
#### 1) Arduino IDE
`$ sudo apt install arduino`
#### 2) Python
`$ sudo apt install python3.6`
#### 3) Numpy
`$ sudo apt install python-numpy`
#### 4) OpenCV
`$ pip install opencv-python`
#### 5) Serial
`$ python -m pip install pyserial`
#### 6) Matplotlib
`$ python -m pip install -U matplotlib`

## How to Run the Code
---
### Arduino UNO
Run `$ arduino` from terminal to open the IDE. Next, click `File > Open` and select `turntable.ino`, located in the turntable folder in this repository.

Connect the microcontroller to the computer via the data sync cable. From the upper toolbar, click `Tools > Board > Arduino UNO` followed by `Tools > Serial Port` where the available port can be selected. In cases where more than one port is proposed, consult [this](https://www.mathworks.com/help/supportpkg/arduinoio/ug/find-arduino-port-on-windows-mac-and-linux.html) document from MathWorks.

One uploads their code by clicking the rightwards arrow. Now onto the Python code!

### Python
#### Part 1: Calibration Preparation

File `camera_test.py` helps one configure their camera for optimal data.

Open `camera_test.py` in an IDE compatible with Python. One connects a webcam or dummy webcam to their PC, and connect the mini projector to the PC via HDMI.

In the main section, modify the webcam port number, width, height and exposure values.

In [17]:
#Put webcam width and height (px) here
webcam_width=960
webcam_height=640

#Put webcam port number here
port_number=3

# Set these two compare exposures, aiming for one below your webcam's noise ceiling. Can be ignored if you are content with your exposure.
exposure1=50
exposure2=75

#Set the cropped area for pixel intensity sample
top_height_px=380
bottom_height_px=420
left_width_px=379
right_width_px=401

Running `python3 camera_test.py` reveals a camera feed and projects an image.
![Changing exposures](README-images/changing_exposures.gif)

One should see the camera feed toggling between exposures, not just by the "Exposure 1" and "Exposure 2" text on the window, but by the camera feed appearing at different brightness levels. If not, Linux users should use Video4Linux to list available controls

`$ v4l2-ctl -d /dev/video0 --list-ctrls`

Not finding `exposure_auto` among the list means the code cannot set the exposure; one can set many DSLR's/dummy webcam's exposure through that camera's in-built menu or explore other ways to use manual exposure. If one is using an actual webcam, the absense of `exposure_auto` poses a high probability that one's camera cannot do this critical function.

One should position their projector so, when a wall is placed along the axis of the turntable and perpendicular to the beams stemming from the turntable base, the black square on the screen is directly above the rotation axis. One should align their camera so the central, red box encases the small black square from the projector. Offsetting anything before calibration will yield a futile lookup table.
![Wall position](README-images/wall.jpeg)

Upon ending the code/while loop, the code with print the average pixel intensity of the cropped area of the camera feed's last frame. Make sure the area is cropped to a region of the projection that is all white pixels; change the exposure values until the average pixel intensity is slightly over 240. The projected fringe patterns span from 50 to 240, hence the average pixel intensity goal.

In [49]:
def exposure_compare(gray, webcam_width, webcam_height, a, b, c, d, e, f):
    # Minimally modified from actual script to work simply in notebook
    avg_color_per_row = np.average(gray[c:d,e:f], axis=0)
    avg_color = np.average(avg_color_per_row, axis=0)
    print('Average pixel intensity (out of 255): '+str(avg_color[1]))

In [37]:
import numpy as np
import cv2

In [52]:
gray = cv2.imread('README-images/sample.png')
exposure_compare(gray, webcam_width, webcam_height, exposure1, exposure2, top_height_px, bottom_height_px, left_width_px, right_width_px)

Average pixel intensity (out of 255): 240.4965909090909


#### Part 2: Calibration

File `curve_calibration.py` takes hundreds of photos at every non-zero, grayscale, uint8 pixel intensity (1 to 255) to compare how a projected pixel intensity is observed by the camera for a lookup table.

**It is crucial that one follow the instructions in *Part 1: Calibration Preparation*, and ensure alignment between the camera and projector before proceeding.**

Open `curve_calibration.py` in an IDE compatible with Python. One consults the `__init__` function of the VideoStreamWidget class to change one's webcam port number (src), width, height and exposure, as validated in `camera_test.py`. Then run `$ python3 curve_calibration.py`. The terminal updates users with which pixel intensity it is sampling, and reveals a calibration curve at the end for one's own judgment on whether it looks like a S, similar to the Sennheiser logo. Assuming one's camera's manual exposure is working as desired, and that they aligned the camera and projector, the curve should look reasonable.

![Calibration Curve Like Sennheiser](README-images/calibration_curve.png)

Ultimately, `curve_calibration.py` uses one image offset 255 times, with a different intensity at the center of the projected screen each time.
![Pics used for calibration](resize_image.png)
By using `camera_test.py` appropriately, the horizontal position of the projected screen's center with respect to the camera is known. Calibration from a gradient yielded better results than projecting a homogeneous image each time.

#### Part 3: 3D Modeling

File `turn_fringe.py` takes images of the subject at different angles as multiple fringe patterns are projected on it.

Open `turn_fringe.py` in an IDE compatible with Python. One must connect their Arduino Uno to their computer, the motor's power supply to a wall outlet, a webcam or dummy webcam to their PC, and the mini projector to the PC via HDMI. In the main section, edit the port number, exposure, camera width, camera height, folder, camera_to_projector_distance and to_wall_distance as appropriate. The last three variables deserve elaboration:
* `folder`: Where every image from this experiment will save. This folder will be formed in the script's directory and override a folder with the same name.
* `camera_to_projector_distance`: The Euclidean distance from the lens of the projector to where the camera lens meets the camera body approximates the length. Measuring in LEGO units, where each circle is 8mm wide, helps. Assign the numerical distance to the variable.
* `to_wall_distance`: The Euclidean distance from halfway between the camera-to-projector line to the End of the turntable. Assign the numerical distance to the variable.
![triangulation](README-images/triangulation.png)

Run `$ python3 turn_fringe.py`. Input fields appear; below is clarification on how one might want to answer each.
* *Number of Steps*: A fringe pattern will be projected at equal offset increments summing from 0 to 2π, with the number of increments determined by one's input. Four steps is often a reasonable choice.
* *Number of Fractions of Rotations*: This input causes the turntable to photograph the object at angle multiples of 360/input. For example, inputting 4 causes the object to be positioned at 90, 180, 270 and 0 degrees.
* *Enter a LOW number of periods along x-axis*: This must be greater than 0 but no greater than 1. If the user puts an invalid answer as the input, the frequency defaults to 0.25.
* *Enter a HIGH number of periods along x-axis*: This must be greater than 1. If the user puts an invalid answer as the input, the frequency defaults to 5.

The procedure will begin by rotating the turntable, then projecting and capturing fringe projections on the subject, and repeat until every pattern at every angle is photographed. The files are then converted into phase maps and disparity maps; the latter are portrayed as point clouds, surface plots, cross-section plots, and plots correlating grayscale color to depth.
![flowchart of sequence of which order images are produced in](README-images/plot-process.png)

If one desires to modify their plots, or make plots from photos they already have without triggering the turntable, camera or projector, they can use the `data_files` function independently. The captured fringe images' folder is the only link between both parts of the script.

It is important to break down the phase shift maps, unwrapped phase maps and depth map if future developers desire to expand upon this work

#### Part 3a: Phase Maps
The following function is a recreation of this equation, showing how to create a phase map for n-steps [[1]](www.osti.gov/biblio/1469785).
![N-Step Formula](README-images/nstep.png)

In [55]:
from PIL import Image

def phaseshift(directory):
    """
    With an N-step fringe projection performed prior, outputting N number of
    webcam images in a directory, this function makes a phase shift map
    accordingly.
    """
    Iarr=np.array([])

    length=Image.open(directory+'/1-0.png').size[0] # gets length of exemplary webcam picture
    height=Image.open(directory+'/1-0.png').size[1] # gets height of exemplary webcam picture

    sin=np.zeros((height,length)) # prepares array to intake sin values
    cos=np.zeros((height,length)) # prepares array to intake cos values

    n=0
    N=len([name for name in os.listdir(directory) if os.path.isfile(os.path.join(directory, name))])
    N=int(N/5)
    for n in range (N): #for every step image in the directory

        # 5 images are taken at each step to minimize artifacts due to photographing a screen
        I0=Image.open(directory+'/'+str(n+1)+'-0.png') # images are titled from 1 to N
        I1=Image.open(directory+'/'+str(n+1)+'-1.png')
        I2=Image.open(directory+'/'+str(n+1)+'-2.png')
        I3=Image.open(directory+'/'+str(n+1)+'-3.png')
        I4=Image.open(directory+'/'+str(n+1)+'-4.png')

        I=np.mean((np.asarray(I0),np.asarray(I1),np.asarray(I2),np.asarray(I3),np.asarray(I4)), axis=0)

        I=np.asarray(I, dtype='float32')
        sin=sin+I*np.sin(2*np.pi*n/N) # Critical math operation from page 27 of https://doi.org/10.1016/j.optlaseng.2018.04.019
        cos=cos+I*np.cos(2*np.pi*n/N) # Critical math operation from aforementioned paper
    phase = np.arctan2(cos,sin) # Critical math; sin a cos parameter order does not compromise depth map

    return phase

#### Part 3b: Unwrapped Map
If not for the `np.around` function, the following would simplify to (high_freq/low_freq)*low. With it, the high and low maps combine to minimize the high map's wrapping artifacts; the higher the ratio, the more dominant the low map is [[2]](https://www.researchgate.net/publication/221111009_Pushing_the_limits_of_digital_imaging_using_structured_illumination).

In [56]:
def unwrap(low, high, low_freq, high_freq):
    un=high+(2*np.pi)*np.around(((high_freq/low_freq)*low-high)/(2*np.pi))
    return un

#### Part 3c: Depth Map
Umtimately, the depth map is a disparity map between images with and without the subject, which its magnitude influenced by certain distances [[3]](https://urldefense.com/v3/__https://www.osapublishing.org/aop/viewmedia.cfm?uri=aop-3-2-128&seq=0__;!!Dq0X2DkFhyF93HkjWTBQKhk!HvhuNI-E_dSXouLmQbDzBBPPxLJmyrVlfaSCM_zGgt6KzDaFiA7JZY26h2_8FmCGzTa3Thekmn-JaIVmmQ$)

In [58]:
def depthmap(to_wall_distance, camera_to_projector_distance, unwrapped, unwrapped_0):
    u=(int(to_wall_distance)/int(camera_to_projector_distance))*(unwrapped-unwrapped_0)
    return u

The array composing the depth map plot is formatted into an .xyz file, which is formatted with the x pixel coordinate, then the y pixel coordinate, then the z disparty:

0, 0, z

0, 1, z

...

0, last_height_pixel, z

1, 0, z

1, 1, z

...

last_width_pixel, 0, 0

...

last_width_pixel, last_height_pixel, 0